In [16]:
import sys
import os
# sys.path.append(os.path.join(os.path.dirname(__file__), 'Cython'))
import torch
from typing import List, Dict, Any
# from server.inference.models import ArchFormRegressor, InitAutoencoder
from autosetup_ml.utils import *
import torch
import torch.nn as nn
import json
import pyvista as pv
import numpy as np
from scipy.spatial.transform import Rotation as R

In [11]:
oas_folder: str = "server"
base_case_id: str = "00000000"
template_case_id: str = "00000000"
# Get base case points
base_oas_file_path = r"D:\projectsd\MaterialsUI_probe\server\00000000.oas"

base_orthoCase = OrthoCase(base_oas_file_path)
base_case_points_t1, base_case_points_t2 = case_landmark_grids(base_orthoCase)     

In [24]:
transforms_dict = {}

for tooth_idx, tooth_id in enumerate(dw_teeth_nums14 + up_teeth_nums14):
    
    tooth_points_t1 = base_case_points_t1[tooth_idx] 
    tooth_points_t2 = base_case_points_t2[tooth_idx] 

    # tooth_points_pred = predictions[tooth_idx]  

    tansform_matrix = calc_transform_matrix_fr_points(tooth_points_t1, tooth_points_t2)
    # print(f"tooth idx, {tooth_idx}, tooth_id{tooth_id}, tansform_matrix \n{tansform_matrix}")
    tooth_transform = get_transform_from_matrix(tansform_matrix)
    # print(f"tooth idx, {tooth_idx}, tooth_id{tooth_id}, tooth tooth_transform \n{tooth_transform}")
    # teeth_transforms.extend([tooth_transform]) 
    transforms_dict[str(tooth_id)] = tooth_transform

# transforms_dict

{'37': {'rotation': {'x': -0.07897485898009346,
   'y': 0.024482178038129722,
   'z': 0.0651932316361102,
   'w': 0.9944412688321137},
  'translation': {'x': -2.525350570678711,
   'y': -2.150221109390259,
   'z': -0.8373681306838989}},
 '36': {'rotation': {'x': -0.13261417891044958,
   'y': 0.02951599425163013,
   'z': 0.06288237733551402,
   'w': 0.9887305458292867},
  'translation': {'x': -2.465452194213867,
   'y': -2.016918182373047,
   'z': -3.4378364086151123}},
 '35': {'rotation': {'x': -0.06078240447189676,
   'y': 0.02627104630533182,
   'z': 0.028144914524605897,
   'w': 0.9974082389969717},
  'translation': {'x': -1.2565460205078125,
   'y': -0.1712055206298828,
   'z': -1.3067063093185425}},
 '34': {'rotation': {'x': -0.03906227783010055,
   'y': -0.04857379307853168,
   'z': -0.051887656410034885,
   'w': 0.9967057721258376},
  'translation': {'x': 4.038527488708496,
   'y': 2.189708709716797,
   'z': -3.457822322845459}},
 '33': {'rotation': {'x': -0.03986357656822751,
 

In [13]:
# plot grids in pyvista

# Plot base_case_points_t1 and base_case_points_t2 as point clouds in pyvista
# Convert to numpy arrays if not already
pts_t1 = np.array(base_case_points_t1).reshape(-1, 3)
pts_t2 = np.array(base_case_points_t2).reshape(-1, 3)

plotter = pv.Plotter()
cloud_t1 = pv.PolyData(pts_t1)
cloud_t2 = pv.PolyData(pts_t2)

plotter.add_points(cloud_t1, color='blue', point_size=10, render_points_as_spheres=True, label='T1')
plotter.add_points(cloud_t2, color='red', point_size=10, render_points_as_spheres=True, label='T2')
plotter.add_legend()
plotter.show()


Widget(value='<iframe src="http://localhost:51584/index.html?ui=P_0x27b534fea50_1&reconnect=auto" class="pyvis…

In [21]:
# Paths
crowns_dir = os.path.abspath(os.path.join("..", "..", "public", "crowns"))
ortho_json_path = os.path.abspath(os.path.join("..", "..", "public", "orthoData.json"))

# Load orthoData.json
with open(ortho_json_path, "r") as f:
    ortho_data = json.load(f)

# Get transforms for Stage 0
stage0 = ortho_data["Staging"][0]
stage92 = ortho_data["Staging"][92]

tooth_transforms = stage0["RelativeToothTransforms"]
tooth_transforms2 = stage92["RelativeToothTransforms"]

# List of tooth IDs present in transforms and crowns folder
tooth_ids = [tid for tid in tooth_transforms.keys() if os.path.exists(os.path.join(crowns_dir, f"{tid}.stl"))]

# PyVista plotter
plotter = pv.Plotter()

for tooth_id in tooth_ids:
    mesh_path = os.path.join(crowns_dir, f"{tooth_id}.stl")
    mesh = pv.read(mesh_path)
    tf = tooth_transforms[tooth_id]
    t = tf["translation"]
    r = tf["rotation"]
    # Convert quaternion to rotation matrix using scipy (x, y, z, w)
    quat = [r["x"], r["y"], r["z"], r["w"]]
    rot = R.from_quat(quat).as_matrix()
    # Apply rotation
    mesh.points = mesh.points @ rot.T
    # Apply translation
    mesh.points += np.array([t["x"], t["y"], t["z"]])
    plotter.add_mesh(mesh, color="white", opacity=0.8, show_edges=False)

plotter.show()

Widget(value='<iframe src="http://localhost:51584/index.html?ui=P_0x27b37231a30_8&reconnect=auto" class="pyvis…

In [ ]:
# Plot teeth with transforms from transforms_dict in blue for comparison
plotter = pv.Plotter()

# Plot orthoData.json transforms (white)
for tooth_id in tooth_ids:
    mesh_path = os.path.join(crowns_dir, f"{tooth_id}.stl")
    mesh = pv.read(mesh_path)
    tf = tooth_transforms[tooth_id]
    t = tf["translation"]
    r = tf["rotation"]
    quat = [r["x"], r["y"], r["z"], r["w"]]
    rot = R.from_quat(quat).as_matrix()
    mesh1 = mesh.copy()
    mesh1.points = mesh1.points @ rot.T
    mesh1.points += np.array([t["x"], t["y"], t["z"]])
    plotter.add_mesh(mesh1, color="white", opacity=0.8, show_edges=False, label="orthoData.json")

    tf_dict = transforms_dict[tooth_id]
    t2 = tf_dict["translation"]
    r2 = tf_dict["rotation"]
    quat2 = [r2["x"], r2["y"], r2["z"], r2["w"]]
    rot2 = R.from_quat(quat2).as_matrix()
    # if tooth_id in ["11", "21", "37", "47"]:
    #     # Get the full transform matrix for mesh2
    #     from scipy.spatial.transform import Rotation as R
    #     # Compose the two rotations
    #     rot1 = R.from_quat([r["x"], r["y"], r["z"], r["w"]])
    #     rot2_ = R.from_quat([r2["x"], r2["y"], r2["z"], r2["w"]])
    #     composed_rot = rot2_ * rot1
    #     composed_mat = composed_rot.as_matrix()
    #     # Compose the translation
    #     t1_vec = np.array([t["x"], t["y"], t["z"]])
    #     t2_vec = np.array([t2["x"], t2["y"], t2["z"]])
    #     composed_t = rot2_.apply(t1_vec) + t2_vec
    #     print(f"{tooth_id} - PyVista composed matrix:\n{composed_mat}")
    #     print(f"{tooth_id} - PyVista composed translation: {composed_t}")
    mesh2 = mesh1.copy()
    mesh2.points = mesh2.points @ rot2.T
    mesh2.points += np.array([t2["x"], t2["y"], t2["z"]])
    plotter.add_mesh(mesh2, color="blue", opacity=0.5, show_edges=False, label="transforms_dict")

plotter.add_legend()
plotter.show()


Widget(value='<iframe src="http://localhost:51584/index.html?ui=P_0x27bb0ddc980_20&reconnect=auto" class="pyvi…